In [11]:
from dlo import *

In [12]:
df = pd.read_csv('/deep/group/aicc-bootcamp/cloud-pollution/data/'\
                'combined_v3_typed_new_composite/COCO_format_cropped/train.csv')
df = df[df['contains_shiptrack']]
image = df.iloc[0]['mask']   # loop over rows eventually
image = np.load(image)      
print(np.count_nonzero(image))
image_proc = np.abs(image.astype(int)).astype(np.uint8)
# Image.fromarray(image_proc).save("dlo_orig.png")

# Skeletonize
thinned = skeleton(image)

# Find contours
# https://github.com/opencv/opencv/blob/4.x/samples/python/contours.py
contours0, hierarchy = cv.findContours(thinned.copy(), mode = cv.RETR_TREE, 
                                        method = cv.CHAIN_APPROX_SIMPLE)
# contours = [cv.approxPolyDP(cnt, 3, True) for cnt in contours0] DONT DO THIS
h, w = thinned.shape[:2]
levels = 0
vis = np.zeros((h, w, 3), np.uint8)
cv.drawContours(vis, contours0, -1, (128,255,255), 1, cv.LINE_AA, 
                hierarchy, abs(levels) )
print(np.count_nonzero(vis))
Image.fromarray(vis).save("dlo_contour_max6.png")

#fit and prune doo segments 
chain_collns = [traverseContour(contour) for contour in contours0]
# draw_chain_collns(chain_collns, h, w)
pruned = [prune(chain_colln) for chain_colln in chain_collns]
merged = merge_all_chains(pruned)

draw_chain_collns(pruned, h, w)


1781
249
(1015, 1354)
1782
(208, 1, 2)
(1, 2)
[32, 8, 157, 11]
(4, 1, 2)
(1, 2)
[0, 0, 3, 1]
(4, 1, 2)
(1, 2)
[0, 0, 3, 1]
[[(array([[480, 822]], dtype=int32), array([[485, 830]]))], [(array([[492, 837]], dtype=int32), array([[500, 842]])), (array([[500, 842]]), array([[508, 848]]))], [(array([[516, 851]], dtype=int32), array([[525, 852]]))], [(array([[530, 854]], dtype=int32), array([[537, 860]])), (array([[537, 860]]), array([[543, 867]])), (array([[543, 867]]), array([[550, 873]])), (array([[550, 873]]), array([[558, 877]])), (array([[558, 877]]), array([[565, 883]])), (array([[565, 883]]), array([[573, 887]])), (array([[573, 887]]), array([[581, 892]])), (array([[581, 892]]), array([[589, 896]])), (array([[589, 896]]), array([[597, 900]])), (array([[597, 900]]), array([[605, 904]])), (array([[605, 904]]), array([[613, 908]])), (array([[613, 908]]), array([[620, 914]])), (array([[620, 914]]), array([[627, 920]]))], [(array([[632, 922]], dtype=int32), array([[624, 914]]))], [(array([

In [22]:
def draw_chain(chain, h ,w, img_path='dlo_segments_merged.png', color=0):
    vis = np.zeros((h, w, 3), np.uint8)
    for segment in chain[0]:
        cv.line(vis, segment[0][0], segment[1][0], (color, 255 - color, 255), 1)
    Image.fromarray(vis).save(img_path)

In [122]:
def merge_two_chains(c1, c2):
    combos = [[[c1[ai], ai], [c2[bi], bi]] for ai in [0,-1] for bi in [0,-1]]

    costs = []
    for combo in combos:
        s1, s2 = combo
        weights = [1,1,1]
        costs.append(merge_cost(s1, s2, weights))

    to_connect = combos[np.argmin(costs)]

    # 4 cases
    ret = []
    if (to_connect[0][-1] == -1) and (to_connect[-1][-1] == 0):
        ret = c1 + c2
    elif (to_connect[0][-1] == 0) and (to_connect[-1][-1] == -1):
        ret = c2 + c1
    elif (to_connect[0][-1] == 0) and (to_connect[-1][-1] == 0):
        ret = c1[::-1] + c2
    elif (to_connect[0][-1] == -1) and (to_connect[-1][-1] == -1):
        ret = c1 + c2[::-1]

    return ret



In [130]:
def merge_all_chains(pruned):
    to_merge = []
    for chain_clcn in pruned:
        for chain in chain_clcn:
            if chain:
                to_merge.append(chain)

        while len(to_merge) > 1:
            c1 = to_merge.pop()
            c2 = to_merge.pop()
            merged = merge_two_chains(c1, c2)
            to_merge.append(merged)
    return to_merge

In [131]:
merge_all_chains(pruned)

[[(array([[491, 834]]), array([[483, 827]])),
  (array([[499, 841]]), array([[491, 834]])),
  (array([[508, 847]], dtype=int32), array([[499, 841]])),
  (array([[527, 852]]), array([[517, 851]])),
  (array([[530, 854]], dtype=int32), array([[537, 860]])),
  (array([[537, 860]]), array([[543, 867]])),
  (array([[543, 867]]), array([[550, 873]])),
  (array([[550, 873]]), array([[558, 877]])),
  (array([[558, 877]]), array([[565, 883]])),
  (array([[565, 883]]), array([[573, 887]])),
  (array([[573, 887]]), array([[581, 892]])),
  (array([[581, 892]]), array([[589, 896]])),
  (array([[589, 896]]), array([[597, 900]])),
  (array([[597, 900]]), array([[605, 904]])),
  (array([[605, 904]]), array([[613, 908]])),
  (array([[613, 908]]), array([[620, 914]])),
  (array([[620, 914]]), array([[627, 920]]))]]

In [133]:

c1 = pruned[0][3]
c2 = pruned[0][-1]


combos = [[[c1[ai], ai], [c2[bi], bi]] for ai in [0,-1] for bi in [0,-1]]

costs = []
for combo in combos:
    s1, s2 = combo
    weights = [1,1,1]
    costs.append(merge_cost(s1, s2, weights))

to_connect = combos[np.argmin(costs)]
# 4 cases
ret = []
if (to_connect[0][-1] == -1) and (to_connect[-1][-1] == 0):
    ret = c1 + c2
elif (to_connect[0][-1] == 0) and (to_connect[-1][-1] == -1):
    ret = c2 + c1
elif (to_connect[0][-1] == 0) and (to_connect[-1][-1] == 0):
    ret = c1[::-1] + c2
elif (to_connect[0][-1] == -1) and (to_connect[-1][-1] == -1):
    ret = c1 + c2[::-1]


print(ret)




[(array([[620, 914]]), array([[627, 920]])), (array([[613, 908]]), array([[620, 914]])), (array([[605, 904]]), array([[613, 908]])), (array([[597, 900]]), array([[605, 904]])), (array([[589, 896]]), array([[597, 900]])), (array([[581, 892]]), array([[589, 896]])), (array([[573, 887]]), array([[581, 892]])), (array([[565, 883]]), array([[573, 887]])), (array([[558, 877]]), array([[565, 883]])), (array([[550, 873]]), array([[558, 877]])), (array([[543, 867]]), array([[550, 873]])), (array([[537, 860]]), array([[543, 867]])), (array([[530, 854]], dtype=int32), array([[537, 860]])), (array([[508, 847]], dtype=int32), array([[499, 841]])), (array([[499, 841]]), array([[491, 834]])), (array([[491, 834]]), array([[483, 827]]))]


In [101]:
# Question: how to define s1/s2 end?
# Currently assuming s1.end = s1[0] & s2.end = s2[1]
# TODO: Define as unattached 

"""
Input: 
s1 [[[endpoint1, endpoint2], index_of_segment]]
"""

def euclidean_cost(s1, s2):
    return np.linalg.norm(s1[0][s1[-1]] - s2[0][s2[-1]])

def dir_cost(s1, s2):
    v_s1 = s1[0][-1] - s1[0][0]
    v_s2 = s2[0][-1] - s2[0][0]
    return float(np.abs(np.arccos(np.inner(-v_s1, v_s2)/(np.linalg.norm(s1[0]) * np.linalg.norm(s2[0])))))

def curv_cost(s1, s2):
    v_s1 = s1[0][-1] - s1[0][0]
    v_s2 = s2[0][-1] - s2[0][0]
    v_s21 = s2[0][s2[-1]] - s1[0][s1[-1]]
    v_s12 = s1[0][s1[-1]] - s2[0][s2[-1]]

    c1 = float(np.abs(np.arccos(np.inner(v_s1, v_s21)/(np.linalg.norm(s1[0]) * np.linalg.norm(v_s21)))))
    c2 = float(np.abs(np.arccos(np.inner(v_s2, v_s12)/(np.linalg.norm(s2[0]) * np.linalg.norm(v_s12)))))

    return np.maximum(c1, c2)

def merge_cost(s1, s2, weights): 
    w_c, w_d, w_e = weights
    return np.sum([w_c * curv_cost(s1,s2), w_d * dir_cost(s1,s2), w_e * euclidean_cost(s1,s2)])

In [77]:
merge_cost(s1, s2, [1,1,1])


22.795885147754007

[[[],
  [],
  [],
  [(array([[530, 854]], dtype=int32), array([[537, 860]])),
   (array([[537, 860]]), array([[543, 867]])),
   (array([[543, 867]]), array([[550, 873]])),
   (array([[550, 873]]), array([[558, 877]])),
   (array([[558, 877]]), array([[565, 883]])),
   (array([[565, 883]]), array([[573, 887]])),
   (array([[573, 887]]), array([[581, 892]])),
   (array([[581, 892]]), array([[589, 896]])),
   (array([[589, 896]]), array([[597, 900]])),
   (array([[597, 900]]), array([[605, 904]])),
   (array([[605, 904]]), array([[613, 908]])),
   (array([[613, 908]]), array([[620, 914]])),
   (array([[620, 914]]), array([[627, 920]]))],
  [],
  [],
  [],
  [(array([[527, 852]]), array([[517, 851]]))],
  [(array([[508, 847]], dtype=int32), array([[499, 841]])),
   (array([[499, 841]]), array([[491, 834]])),
   (array([[491, 834]]), array([[483, 827]]))]],
 [[]],
 [[]]]